In [1]:
import pandas as pd
import numpy as np
import pandas.io.sql as psql
from IPython.display import HTML
import IPython
import pymongo
import os
from pandas.io.json import json_normalize
import dateutil.relativedelta
import datetime
pd.set_option('display.max_rows', 1000)
import yaml
import json
from datetime import date
import re
from collections import Counter
import mysql.connector
from forex_python.converter import CurrencyRates
today = date.today()
#d1 = today.strftime("%Y-%m-%d")
d1='2023-03-26'



In [2]:
######Databases Connection

def connect_mongohist():
    ip = "xxxxxxxxx"
    client = pymongo.MongoClient(ip)['xxxxxxx']
    return client
client = connect_mongohist()


In [3]:
#####Functions for Event details

def traded_ids(bookie):
    bet_result_collection = client.betOrders
    df = pd.json_normalize(list(bet_result_collection.find({"eventId":{"$regex":d1,"$options" : "i"},"bookie":bookie})))
    if df.empty:
        return df
    else:
        return list(set(df['eventId']))
    
def get_todays_bfevents():
    bf_events = []
    event_ids = pd.json_normalize(list(client.xxxxx.find({"event_id": {"$in": [re.compile(f"{d1}#cr#")]}}))).event_id.tolist()
    bf_events = pd.json_normalize(list(client.xxxxx.find({"eventId": {"$in": event_ids}})))
    if(bf_events.empty == True):
        return bf_events
    else:
        return bf_events.eventId.unique().tolist()

def get_events_details(eventid):
    bet_result_collection = client.xxxxxx
    df = pd.json_normalize(list(xxxxxx.find({'nativeContestId':eventid})))
    return df   

def get_results(bo):
    chased=0
    results=[]
    for i in range(len(bo)):
        home_team=bo.iloc[i]['nativeHomeTeamId']
        away_team=bo.iloc[i]['nativeAwayTeamId']
        first_batter=bo.iloc[i]['innings'][0]['nativeBattingTeamId']
        if(bo.iloc[i]['innings'][1]['runs'] > bo.iloc[i]['innings'][0]['runs']):
            chased=1
        if(chased==1 and first_batter==home_team):
            result='a'
        elif(chased==1 and first_batter==away_team):
            result='h'
        elif(chased==0 and first_batter==home_team):
            result='h'
        else:
            result='a'
        if(bo.iloc[i]['innings'][1]['runs'] == bo.iloc[i]['innings'][0]['runs']):
            result='t'
        results.append(result)
    return results


In [4]:
######Misc functions

def get_bet_orders(event_id):
    bet_result_collection = client.xxxxx
    df = pd.json_normalize(list(bet_result_collection.find({"eventId":event_id})))
    return df


def get_bet_results(event_id):
    bet_result_collection = client.xxxxx
    df = pd.json_normalize(list(bet_result_collection.find({"eventId":event_id,"betStatus":"successs"})))
    return df

def bet_Type(bet):
    return bet[10:]

def compute_pnl(res,bet,stake,price):
    if(res=='t'):
        pnl='tied'
    elif(res==bet_Type(bet)[0] and len(bet_Type(bet))>1):   #loosing lay
        pnl=-((stake*price)-stake) 
    elif(res==bet_Type(bet)[0] and len(bet_Type(bet))==1): #winning
        pnl=(stake*price)-stake
    elif(res!=bet_Type(bet)[0] and len(bet_Type(bet))>1):  #winning lay
        pnl=stake
    else:                                                  #losing
        pnl=-stake
    return pnl


def find_if_betruns_scored(event,over,innings,runs):
    match_data=get_events_details(event)
    scored='no'
    totalOver=match_data.iloc[0]['innings'][innings-1]['overs'][-1]['over'] + 1
    for i in range(len(match_data)):
        if(over<=totalOver):
            scored_runs=match_data.iloc[i]['innings'][innings-1]['overs'][over-1]['cumulativeRuns']
        else:
            scored_runs=match_data.iloc[i]['innings'][innings-1]['overs'][totalOver-1]['cumulativeRuns']
    if(runs/4 > scored_runs):
        scored='yes'
        return scored
    else:
        return scored

def bet_count(eventid,bookie):
    bet_count=['total','success']
    bet_result_collection = client.xxxxx
    df = pd.json_normalize(list(bet_result_collection.find({"eventId":eventid,"bookie":bookie})))#"betStatus":"successs"
    return dict(Counter(df['betStatus']))


In [5]:
#####Functions for broker commissions and Turnover

def client1_postComms(pnl):
    if pnl > 0:
        comms= pnl * -0.02
    else:
        comms= pnl * 0.01
    return pnl+comms

def client2_postComms(pnl):
    if pnl > 0:
        comms= pnl * -0.01
    else:
        comms= pnl * 0.01
    return pnl+comms

def compute_turnOver(bet,stake,price):
    if(len(bet_Type(bet))>1):   #lay bet
        to=(stake*price)-stake
    elif(len(bet_Type(bet))==1):
        to=stake                #back bet
    return to

def convertCurr(amount):
    cr = CurrencyRates()
    return cr.convert('USD','GBP',amount)
    return amount*0.8

In [7]:
#####Functions for PnL calculation

def generate_client1_pnl(events,dash_frame):
    for i in events:
        if('successs' not in bet_count(i,'client1').keys()):
            events.remove(i)
    for i in events:
        bo=get_bet_orders(i)
        br=get_bet_results(i)
        event_details=get_events_details(i)
        if(event_details.empty or len(event_details.innings[0]) < 2):
            continue
        result=get_results(event_details)
        result=''.join(result)
        merge = pd.merge(br,bo, left_on="internalOrderId", right_on="trace_id")
        merge = merge[['betType','account','stake_x','price_x']]
        merge['stake_x'] = merge['stake_x']/100
        merge=merge.loc[merge['account']=='broker3']
        pnl,to=[],[]
        for j in range(len(merge)):
            pnl.append(compute_pnl(result,merge['betType'].values[j],merge['stake_x'].values[j],merge['price_x'].values[j]))

        for k in range(len(merge)):
            to.append(compute_turnOver(merge['betType'].values[k],merge['stake_x'].values[k],merge['price_x'].values[k]))
        merge['PnL'] = pnl
        merge['Turnover'] = to
        totalPnL=merge['PnL'].sum()
        totalTO=merge['Turnover'].sum()
        PnL_after_comms = client1_postComms(totalPnL)

        bet_info = bet_count(i,'client1') 
        dash_frame=dash_frame.append({'event':i,'broker':'client1','pnl_gbp':totalPnL,'pnl_usd':np.nan,'pnl_post_comms':PnL_after_comms,'turnover':totalTO,'total_bets':sum(bet_info.values()),'success_bets':bet_info['successs']},ignore_index=True)
        dash_frame= dash_frame[dash_frame['pnl_gbp'] != 0]
    return dash_frame


def generate_client2_pnl(events,dash_frame): 
    for i in events:
        if('successs' not in bet_count(i,'client2').keys()):                    #removing ids with only failed bets
            events.remove(i)
    for i in events:
        bo=get_bet_orders(i)
        br=get_bet_results(i)
        event_details=get_events_details(i)
        if(event_details.empty or event_details.innings[0]) < 2):
            continue
        result=get_results(event_details)
        result=''.join(result)
        merge = pd.merge(br,bo, left_on="internalOrderId", right_on="trace_id")
        merge = merge[['betType','account','stake_x','price_x']]
        merge['stake_x'] = merge['stake_x']/100
        merge=merge.loc[merge['account']=='broker1']
        pnl,to=[],[]
        for j in range(len(merge)):
            pnl.append(compute_pnl(result,merge['betType'].values[j],merge['stake_x'].values[j],merge['price_x'].values[j]))

        for k in range(len(merge)):
            to.append(compute_turnOver(merge['betType'].values[k],merge['stake_x'].values[k],merge['price_x'].values[k]))
        merge['PnL'] = pnl
        merge['Turnover'] = to
        totalPnL=merge['PnL'].sum()
        totalTO=merge['Turnover'].sum()
        PnL_after_comms = client2_postComms(convertCurr(totalPnL))

        bet_info = bet_count(i,'client2') 
        dash_frame=dash_frame.append({'event':i,'broker':'client2','pnl_gbp':convertCurr(totalPnL),'pnl_usd':totalPnL,'pnl_post_comms':PnL_after_comms,'turnover':convertCurr(totalTO),'total_bets':sum(bet_info.values()),'success_bets':bet_info['successs']},ignore_index=True)
        dash_frame= dash_frame[dash_frame['pnl_gbp'] != 0]  
    return dash_frame

CLIENT = pymongo.MongoClient("xxxxxxxx", readPreference="nearest")["xxxxxx"]
INSTRUCTION_COLLECTION = CLIENT["xxxxxxx"]
INSTRUCTION_FIELDS = {
    "_id": 0,
    "eventId": 1,
    #"strategyName": 1,
    "placeInstructions.id": 1,
    #"placeInstructions.stake": 1,
    "placeInstructions.selectionType": 1,
    #"placeInstructions.price": 1,
    "placeInstructions.side": 1,
    "placeInstructions.marketIdentifier":1,
}


def get_instructions(eid):
    agg_query = [
        {"$match": {"eventId": eid}},
        {"$unwind": "$placeInstructions"},
        {"$project": INSTRUCTION_FIELDS},
    ]
    instructions = pd.json_normalize(
        list(INSTRUCTION_COLLECTION.aggregate(agg_query))
    )
    return instructions

REPORT_FIELDS = {
    "_id": 0,
    "reports.placeInstructionId": 1,
    "reports.stake": 1,
    "reports.price": 1,
}
MATCHED_REPORT_COLLECTION = CLIENT["xxxxxxx"]

def get_matched_reports(eid):
    report_agg = [
        {"$match": {"eventId": eid}},
        {"$unwind": "$reports"},
        {"$project": REPORT_FIELDS},
    ]
    reports = pd.json_normalize(
        list(MATCHED_REPORT_COLLECTION.aggregate(report_agg))
    )
    return reports

def calculate_BFpnl(event,res,bet,side,stake,price,selection):
    stake=float(stake[4:])
    if(res=='t'):
        return 'tied'
    elif (selection == 'o'):
        overs=bet[3][6:]
        innings=bet[2][8:]
        runs=bet[4][5:]
        #print(find_if_betruns_scored(event,int(overs),int(innings),int(runs)))
        if((find_if_betruns_scored(event,int(overs),int(innings),int(runs))=='yes')):
           #print(side)
           if(side=='LAY'):      #Win 
               return stake
           else:
               return -stake     #losing lay
        else:
           #print(side)
           if(side=='LAY'):
               return -stake
           else:
               return stake 
    else:
        if(res==selection and side=='LAY'):   #loosing Lay
            return -((stake*price)-stake)
        elif(res==selection and side=='BACK'):
            return (stake*price)-stake
        elif(res!=selection and side=='LAY'):  #Win Lay
            return stake
        else:
            return -stake
    
def generate_bf_pnl(events,dash_frame): 
    for event_id in events:
        bf_pnl=[]
        event_details=get_events_details(event_id)
        if(event_details.empty or len(event_details.innings[0]) < 2):
            continue
        else:
            result=get_results(event_details)
            result=''.join(result)
            instructions = get_instructions(event_id)
            matched_reports = get_matched_reports(event_id)
            merg=instructions.merge(matched_reports, how="right", left_on="placeInstructions.id", right_on="xxxx.placeInstructionId")
            #print(merg)
            for i in range(len(merg)):
                bf_pnl.append(calculate_BFpnl(merg['eventId'].values[i],result,list(merg['placeInstructions.marketIdentifier'].values[i].split(',')),merg['placeInstructions.side'].values[i],merg['xxxxx.stake'].values[i],merg['xxxxx.price'].values[i],merg['placeInstructions.selectionType'].values[i]))
            bet_count=len(bf_pnl)
            pnl=sum(bf_pnl)
            dash_frame=dash_frame.append({'event':event_id,'broker':'bf','pnl_gbp':pnl,'pnl_usd':np.nan,'pnl_post_comms':(pnl*-0.4)+pnl,'turnover':np.nan,'total_bets':bet_count,'success_bets':np.nan},ignore_index=True)
            dash_frame= dash_frame[dash_frame['pnl_gbp'] != 0]
    return dash_frame


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 90)

In [ ]:
######Final callable function

def generate_mongo_rec():
    dash_frame=pd.DataFrame(columns=['event','broker','pnl_gbp','pnl_usd','pnl_post_comms','turnover','total_bets','success_bets'])
    client1Bo=traded_ids('client1')
    client2Bo=traded_ids('client2')
    days_events=get_todays_bfevents()
    dash_frame=generate_client1_pnl(client1Bo,dash_frame)
    dash_frame=generate_client2_pnl(client2Bo,dash_frame)
    dash_frame=generate_bf_pnl(days_events,dash_frame)
    cols=['pnl_gbp','pnl_post_comms','turnover']
    dash_frame[cols]=dash_frame[cols].round(0)
    dash_frame=dash_frame.drop_duplicates()
    return dash_frame


In [ ]:
#####Functions for SQL connection

def connection():
    conn = mysql.connector.connect(host="xxxxx",
                                       database="xxxxx",
                                       user="xxxxxx",
                                       password="xxxxxxx",
                                       connect_timeout=1000)
    
    return conn
conn=connection()

def sqlquery(query):
    conn = connection()
    df = pd.read_sql(sql=query, con=conn)
    conn.close()
    return df

In [ ]:
#####Functions for extraction of rec from AD1

def pnl_from_ad1(date):
    sql="select EventId as 'event',orderBookie as 'broker',sum(ProfitGbp) as 'pnl_gbp',sum(Profit)/100 as 'pnl_usd' from xxxx where orderdate = '{0}' and Sport = 'cr' group by EventId , Bookie order by EventId;".format(date)
    df = pd.read_sql_query(sql, conn)
    return df

def sqlrec(sql_df):
    comms,turnover,total_bets,success_bets=[],[],[],[]
    for i in range(len(sql_df)):
        event=sql_df.iloc[i]['event']
        bookie=sql_df.iloc[i]['broker']
        temp_rec=dash_frame.query('broker==@bookie and event==@event')
        if(bookie=='client1'):
            comms.append(client1PnL_postComms(sql_df.iloc[i]['pnl_gbp']))
        elif(bookie=='client2'):
            comms.append(client2PnL_postComms(sql_df.iloc[i]['pnl_gbp']))
        else:
            comms.append(temp_rec.iloc[0]['pnl_post_comms'])
        turnover.append(temp_rec.iloc[0]['turnover'])
        total_bets.append(int(temp_rec.iloc[0]['total_bets']))
        success_bets.append(temp_rec.iloc[0]['success_bets'])
    sql_df['pnl_post_comms']=comms
    sql_df['turnover']=turnover   
    sql_df['total_bets']=total_bets
    sql_df['success_bets']=success_bets
    return sql_df

def generate_sql_rec(date):
    ad_rec = pnl_from_ad1(date)
    ad_rec=sqlrec(ad_rec)
    return ad_rec



In [ ]:
dash_frame=generate_mongo_rec()
dash_frame

In [ ]:
hist_dash_frame=generate_sql_rec(d1)
hist_dash_frame